# Default parametrelerle denenen modellerin hyper parametre tuningi ile tekrar sonuçlarının kontrol edilmesi

## Kütüphanelerin import edilmesi

In [38]:
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import metrics

from tensorflow.keras.utils import to_categorical

## GridSearch İşlemi Uygulanacak Makine Öğrenimi Modellerinin Tanımlanması

### KNN Grid Search Uygulanması ve optimum parametrelerin bulunması

In [39]:
df = pd.read_csv('data/clustered_data.csv').drop(['id', 'player_name', 'position'], axis=1)
X = df.drop(['classes'], axis = 1).values
y = df['classes']
n_cols = len(np.unique(y))

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(2)
X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.36, random_state=14)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

knn = KNeighborsClassifier()
decision_tree = DecisionTreeClassifier()

k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy',
                    return_train_score=False, verbose=1)


grid_search=grid.fit(X_train, y_train)

print(grid_search.best_params_)
n_neighbors = grid_search.best_params_['n_neighbors']


Fitting 10 folds for each of 30 candidates, totalling 300 fits
{'n_neighbors': 3}


In [3]:
accuracy = grid_search.best_score_ *100
print("Parametre tuningi sonucu çıkan optimum başarım : {:.2f}%".format(accuracy) )

Parametre tuningi sonucu çıkan optimum başarım : 98.14%


In [55]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

knn = KNeighborsClassifier(n_neighbors=n_neighbors)

knn.fit(X_train, y_train)

y_pred=knn.predict(X_test) 

test_accuracy=accuracy_score(y_test,y_pred)*100
test_precision = precision_score(y_test, y_pred, pos_label='positive', average='weighted') * 100
test_recall = recall_score(y_test, y_pred, pos_label='positive', average='weighted') * 100
test_f1 = f1_score(y_test, y_pred, pos_label='positive', average='weighted') * 100
acc_knn = np.array([grid_search.best_params_['n_neighbors'], test_accuracy, test_precision, test_recall, test_f1]).reshape(1,-1)
df_acc_knn = pd.DataFrame(acc_knn, columns=['n_neighbors', 'acc', 'precision', 'recall', 'f1'])
df_acc_knn.to_csv('knn_acc_recall_f1_precision.csv')
print("Bu parametrenin test seti üzerindeki başarısı : {:.2f}%".format(test_accuracy) )

Bu parametrenin test seti üzerindeki başarısı : 98.12%


c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [56]:
cm = confusion_matrix(
    y_test.argmax(axis=1), y_pred.argmax(axis=1))
plt.figure(figsize=(12, 12))
sns.heatmap(pd.DataFrame(cm),
                        annot=True, cmap="YlGnBu", fmt='g')
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

plt.savefig('knn_optimum_parameters_cm.png')
plt.clf()


<Figure size 864x864 with 0 Axes>

### Decision Tree GridSearch Uygulanması ve optimum parametrelerin bulunması

In [57]:
df = pd.read_csv('data/clustered_data.csv').drop(['id', 'player_name', 'position'], axis=1)
X = df.drop(['classes'], axis = 1).values
y = df['classes']
n_cols = len(np.unique(y))

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(2)
X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.36, random_state=14)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

decision_tree = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [
    4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 120, 150]}

grid = GridSearchCV(decision_tree, param_grid, cv=10, scoring='accuracy',
                    return_train_score=False, verbose=1)


grid_search = grid.fit(X_train, y_train)

print(grid_search.best_params_)
criterion = grid_search.best_params_['criterion']
max_depth = grid_search.best_params_['max_depth']

Fitting 10 folds for each of 36 candidates, totalling 360 fits
{'criterion': 'gini', 'max_depth': 120}


In [58]:
accuracy = grid_search.best_score_ *100
print("Parametre tuningi sonucu çıkan optimum başarım : {:.2f}%".format(accuracy) )

Parametre tuningi sonucu çıkan optimum başarım : 97.80%


In [59]:
from sklearn.metrics import confusion_matrix, accuracy_score

decision_tree = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)

decision_tree.fit(X_train, y_train)

y_pred=decision_tree.predict(X_test) 

test_accuracy=accuracy_score(y_test,y_pred)*100
test_precision = precision_score(y_test, y_pred, pos_label='positive', average='weighted') * 100
test_recall = recall_score(y_test, y_pred, pos_label='positive', average='weighted') * 100
test_f1 = f1_score(y_test, y_pred, pos_label='positive', average='weighted') * 100
acc_dt = np.array([grid_search.best_params_['criterion'], grid_search.best_params_['max_depth'], test_accuracy, test_precision, test_recall, test_f1]).reshape(1,-1)
df_acc_dt = pd.DataFrame(acc_dt, columns=['criterion','max_depth','acc', 'precision', 'recall', 'f1'])
df_acc_dt.to_csv('dt_acc_recall_f1_precision.csv')
print("Bu parametrenin test seti üzerindeki başarısı : {:.2f}%".format(test_accuracy) )

Bu parametrenin test seti üzerindeki başarısı : 97.89%


c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [60]:
cm = confusion_matrix(
    y_test.argmax(axis=1), y_pred.argmax(axis=1))
plt.figure(figsize=(12, 12))
sns.heatmap(pd.DataFrame(cm),
                        annot=True, cmap="YlGnBu", fmt='g')
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

plt.savefig('decision_tree_optimum_parameters_cm.png')
plt.clf()

<Figure size 864x864 with 0 Axes>

### Ann ile çeşitli batch-size değerlerinin denenmesi
[8, 16, 32, 64, 128, 256] değerleriyle denemeler yapılmış ve test seti üzerindeki başarıları kıyaslanmıştır.

In [97]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn import metrics

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras import callbacks
import tensorflow as tf
import keras

def createModel(n_cols):

    model = keras.Sequential([

        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(units=32, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=8, activation='relu'),
        keras.layers.Dense(units=n_cols, activation='softmax'),
    ]
    )

    optim = Adam(lr=0.001)
    model.compile(optimizer=optim, loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [101]:
def ann_model(X_train, y_train, X_test, y_test, batch_size):
        early_stopping = callbacks.EarlyStopping(
        min_delta=0.00001,
        patience=6,
        restore_best_weights=True)
        model = createModel(n_cols=n_cols)

        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1000, callbacks=[
                        early_stopping], validation_split=0.2, verbose = 2)
        model.save('model/model_sc_pca_batch_size_'+str(batch_size)+'.h5')
        plt.figure(figsize=(12, 12))
        plt.subplot(211)
        plt.title('Loss')
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='test')
        plt.legend()
        
        plt.subplot(212)
        plt.title('Accuracy')
        plt.plot(history.history['accuracy'], label='train')
        plt.plot(history.history['val_accuracy'], label='test')
        plt.legend()
        plt.savefig('ann_default_parameters_acc_loss_batch_size_'+str(batch_size)+'.png')
        plt.clf()
        y_pred = model.predict(X_test)
        cnf_matrix = metrics.confusion_matrix(
                y_test.argmax(axis=1), y_pred.argmax(axis=1))
        plt.figure(figsize=(12, 12))
        sns.heatmap(pd.DataFrame(cnf_matrix),
                        annot=True, cmap="YlGnBu", fmt='g')
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')

        plt.savefig('ann_default_parameters_cm_batch_size_'+str(batch_size)+'.png')
        plt.clf()
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)

        acc = (TP + TN) / (TP + FP + FN + TN)
        sen = TP / (TP + FN)
        pre = TP / (TP + FP)
        spe = TN / (TN + FP)
        f1 = (2 * (pre * sen)) / (pre + sen)

        acc_scores_ann.append([batch_size, np.mean(acc), np.mean(sen), np.mean(pre), np.mean(spe),np.mean(f1)])

        print("batch_size {} -> Bu parametrenin test seti üzerindeki başarısı : {:.2f}%".format(batch_size, test_accuracy))

        

In [102]:
acc_scores_ann = []
df = pd.read_csv('data/clustered_data.csv').drop(['id', 'player_name', 'position'], axis=1)
X = df.drop(['classes'], axis = 1).values
y = df['classes']
n_cols = len(np.unique(y))

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(2)
X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.36, random_state=14)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [103]:
batches = [8, 16, 32, 64, 128, 256]
for i in batches:
    ann_model(X_train, y_train, X_test, y_test, i)

acc_scores_ann = np.array(acc_scores_ann)
acc_scores_ann

Epoch 1/1000


c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


473/473 - 5s - loss: 1.2772 - accuracy: 0.4956 - val_loss: 0.8716 - val_accuracy: 0.6571 - 5s/epoch - 11ms/step
Epoch 2/1000
473/473 - 4s - loss: 0.8175 - accuracy: 0.7056 - val_loss: 0.5311 - val_accuracy: 0.8646 - 4s/epoch - 8ms/step
Epoch 3/1000
473/473 - 4s - loss: 0.5591 - accuracy: 0.8128 - val_loss: 0.3854 - val_accuracy: 0.8624 - 4s/epoch - 8ms/step
Epoch 4/1000
473/473 - 4s - loss: 0.4536 - accuracy: 0.8472 - val_loss: 0.2600 - val_accuracy: 0.9228 - 4s/epoch - 8ms/step
Epoch 5/1000
473/473 - 3s - loss: 0.3932 - accuracy: 0.8605 - val_loss: 0.3250 - val_accuracy: 0.8899 - 3s/epoch - 7ms/step
Epoch 6/1000
473/473 - 3s - loss: 0.3636 - accuracy: 0.8782 - val_loss: 0.2211 - val_accuracy: 0.9090 - 3s/epoch - 7ms/step
Epoch 7/1000
473/473 - 3s - loss: 0.3423 - accuracy: 0.8896 - val_loss: 0.1802 - val_accuracy: 0.9460 - 3s/epoch - 6ms/step
Epoch 8/1000
473/473 - 2s - loss: 0.2908 - accuracy: 0.9110 - val_loss: 0.1692 - val_accuracy: 0.9312 - 2s/epoch - 5ms/step
Epoch 9/1000
473/473

c:\Users\bugra\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


237/237 - 3s - loss: 1.4547 - accuracy: 0.4355 - val_loss: 1.0479 - val_accuracy: 0.5884 - 3s/epoch - 11ms/step
Epoch 2/1000
237/237 - 2s - loss: 0.9476 - accuracy: 0.6664 - val_loss: 0.5932 - val_accuracy: 0.7947 - 2s/epoch - 7ms/step
Epoch 3/1000
237/237 - 2s - loss: 0.6366 - accuracy: 0.7628 - val_loss: 0.3986 - val_accuracy: 0.8741 - 2s/epoch - 7ms/step
Epoch 4/1000
237/237 - 2s - loss: 0.5322 - accuracy: 0.8242 - val_loss: 0.2914 - val_accuracy: 0.9376 - 2s/epoch - 8ms/step
Epoch 5/1000
237/237 - 2s - loss: 0.4403 - accuracy: 0.8501 - val_loss: 0.2487 - val_accuracy: 0.9450 - 2s/epoch - 9ms/step
Epoch 6/1000
237/237 - 2s - loss: 0.3806 - accuracy: 0.8666 - val_loss: 0.2417 - val_accuracy: 0.9397 - 2s/epoch - 7ms/step
Epoch 7/1000
237/237 - 2s - loss: 0.3587 - accuracy: 0.8848 - val_loss: 0.2213 - val_accuracy: 0.9397 - 2s/epoch - 8ms/step
Epoch 8/1000
237/237 - 1s - loss: 0.3446 - accuracy: 0.8809 - val_loss: 0.1829 - val_accuracy: 0.9450 - 1s/epoch - 5ms/step
Epoch 9/1000
237/237

...

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

In [111]:
acc_scores_df = pd.DataFrame(acc_scores_ann, columns=['batch_size', 'acc', 'sen', 'pre', 'spe','f1'])
acc_scores_df.dropna().to_csv('ann_test_ac_df.csv')
acc_scores_df.dropna()

,batch_size,acc,sen,pre,spe,f1
0,8.0,0.992548,0.870284,0.928784,0.995800,0.885629
1,16.0,0.993451,0.937167,0.905428,0.996434,0.915342
3,64.0,0.989763,0.837616,0.880790,0.994337,0.849902
4,128.0,0.993602,0.878836,0.920014,0.996478,0.888230


Yukarıdaki başarılar göz önünde bulundurulduğunda ann optimum test başarısını %99,36'yla 128 batch_size değerinde vermiştir. </br>
Tüm değerler csv dosyası olarak kaydedilmiştir.